# Example profiling of `empymod`

In [ ]:
import numpy as np
from line_profiler import LineProfiler
%load_ext memory_profiler

import empymod
from empymod import transform, kernel, model, utils, filters

## Profiler `do_profile`

The function `do_profile` is taken from:
  - Blog : https://zapier.com/engineering/profiling-python-boss/
  - GitHub: https://github.com/zapier/profiling-python-like-a-boss/blob/master/timers.py.

License:

Copyright (c) 2013, Zapier Inc. All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

   - Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
   - Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
   - Neither the name of the Zapier Inc. nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [ ]:
def do_profile(follow=[]):
    """
    Simply decorate an expensive function and optionally
    pass other functions to follow. For example:
        def get_number():
            for x in xrange(5000000):
                yield x
        @do_profile(follow=[get_number])
        def expensive_function():
            for x in get_number(5000000):
                i = x ^ x ^ x
            return 'some result!'
    """
    def inner(func):
        def profiled_func(*args, **kwargs):
            try:
                profiler = LineProfiler()
                profiler.add_function(func)
                for f in follow:
                    profiler.add_function(f)
                profiler.enable_by_count()
                return func(*args, **kwargs)
            finally:
                profiler.print_stats()
        return profiled_func
    return inner

### Define Model

In [ ]:
src   = [0, 0, 400]
rec   = [np.arange(1,1001)*10, np.arange(1,1001)*3, 400]
depth = [ 0, 200, 500, 600, 1100, 1300, 1800, 2300, 3000]
res   = [1e20, 4, 10, 20, 1/.2, 1/.3, 1/.4, 1/.5, 1/.6, 1/.7]
aniso  = [.5, 4, 2, 1.5, 3, .25, 2, .5, 1, 2]
epermH = [1, 80, 5, 10, 20, 3, 20, 5, 8, 10]
epermV = [1, 80, 5, 20, 10, 8, 10, 5, 6, 5]
mpermH = [1, 1, 1.0001, 1, 1, 1.00001, 1, 1, 1, 1]
mpermV = [1, 1, 1, 1.0001, 1.0001, 1, 1, 1, 1, 1]
freq = np.arange(1,21)

## Profile

Activate or adjust depending on what you want to profile

In [ ]:
#@do_profile(follow=[model.frequency, hankel.fht, kernel.wavenumber])
#@do_profile(follow=[utils.check_ab, utils.check_param, utils.check_spatial, utils.check_frequency])

@do_profile(follow=[transform.fht, kernel.greenfct, kernel.reflections, kernel.fields])
def test_function():
    out = model.frequency(src=src, rec=rec, depth=depth, res=res, ab=11, freq=freq,
                          ht='FHT', htarg=filters.key_401_2009(),
                          verb=1, xdirect=False, opt='spline',
                          aniso=aniso, epermH=epermH, epermV=epermV, mpermH=mpermH, mpermV=mpermV)
result = test_function()

## Memory and other tests

In [ ]:
opt = 'spline'
HT='FHT'

print('timeit')
%timeit model.frequency(src=src, rec=rec, depth=depth, res=res, aniso=aniso, freq=freq, ab=11, ht=HT, verb=0, xdirect=False, epermH=epermH, epermV=epermV, mpermH=mpermH, mpermV=mpermV, opt=opt)

print('\nmemit')
%memit model.frequency(src=src, rec=rec, depth=depth, res=res, aniso=aniso, freq=freq, ab=11, ht=HT, verb=0, xdirect=False, epermH=epermH, epermV=epermV, mpermH=mpermH, mpermV=mpermV, opt=opt)

print('\nmprun')
%mprun -f kernel.greenfct model.frequency(src=src, rec=rec, depth=depth, res=res, aniso=aniso, freq=freq, ab=11, ht=HT, verb=0, xdirect=False, epermH=epermH, epermV=epermV, mpermH=mpermH, mpermV=mpermV, opt=opt)

print('\nprun')
%prun -l 10 model.frequency(src=src, rec=rec, depth=depth, res=res, aniso=aniso, freq=freq, ab=11, ht=HT, verb=0, xdirect=False, epermH=epermH, epermV=epermV, mpermH=mpermH, mpermV=mpermV, opt=opt)
